In [209]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from dateutil.parser import parse
import datetime
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [210]:
interviews = pd.read_csv("data/raw/Interview.csv")

In [211]:
interviews.head()

,Date of Interview,Client name,Industry,Location,Position to be closed,Nature of Skillset,Interview Type,Name(Cand ID),Gender,Candidate Current Location,...,Are you clear with the venue details and the landmark.,Has the call letter been shared,Expected Attendance,Observed Attendance,Marital Status,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 1,Male,Chennai,...,Yes,Yes,Yes,No,Single,NaN,NaN,NaN,NaN,NaN
1,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 2,Male,Chennai,...,Yes,Yes,Yes,No,Single,NaN,NaN,NaN,NaN,NaN
2,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 3,Male,Chennai,...,NaN,NaN,Uncertain,No,Single,NaN,NaN,NaN,NaN,NaN
3,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 4,Male,Chennai,...,Yes,Yes,Uncertain,No,Single,NaN,NaN,NaN,NaN,NaN
4,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 5,Male,Chennai,...,Yes,Yes,Uncertain,No,Married,NaN,NaN,NaN,NaN,NaN


In [212]:
interviews.tail()

,Date of Interview,Client name,Industry,Location,Position to be closed,Nature of Skillset,Interview Type,Name(Cand ID),Gender,Candidate Current Location,...,Are you clear with the venue details and the landmark.,Has the call letter been shared,Expected Attendance,Observed Attendance,Marital Status,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
1229,07.05.2016,Pfizer,Pharmaceuticals,Chennai,Niche,Biosimiliars,Scheduled,Candidate 1230,Male,Chennai,...,Yes,Yes,Yes,Yes,Single,NaN,NaN,NaN,NaN,NaN
1230,06.05.2016,Pfizer,Pharmaceuticals,Chennai,Niche,Biosimiliars,Scheduled,Candidate 1231,Male,Chennai,...,Yes,Yes,Yes,Yes,Married,NaN,NaN,NaN,NaN,NaN
1231,06.05.2016,Pfizer,Pharmaceuticals,Chennai,Niche,generic drugs – RA,Scheduled,Candidate 1232,Male,Chennai,...,Yes,Yes,Yes,Yes,Single,NaN,NaN,NaN,NaN,NaN
1232,06.05.2016,Pfizer,Pharmaceuticals,Chennai,Niche,generic drugs – RA,Scheduled,Candidate 1233,Female,Chennai,...,NaN,NaN,Uncertain,Yes,Single,NaN,NaN,NaN,NaN,NaN
1233,NaN,﻿﻿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
#the last row is redundant, drop it
interviews = interviews.drop([1233])

#last 5 columns are empty, drop them
interviews = interviews.drop(["Name(Cand ID)", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25", "Unnamed: 26", "Unnamed: 27"], axis = 1)

In [214]:
interviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1232
Data columns (total 22 columns):
Date of Interview                                                                                     1233 non-null object
Client name                                                                                           1233 non-null object
Industry                                                                                              1233 non-null object
Location                                                                                              1233 non-null object
Position to be closed                                                                                 1233 non-null object
Nature of Skillset                                                                                    1233 non-null object
Interview Type                                                                                        1233 non-null object
Gender                          

In [215]:
interviews.columns

Index(['Date of Interview', 'Client name', 'Industry', 'Location',
       'Position to be closed', 'Nature of Skillset', 'Interview Type',
       'Gender', 'Candidate Current Location', 'Candidate Job Location',
       'Interview Venue', 'Candidate Native location',
       'Have you obtained the necessary permission to start at the required time',
       'Hope there will be no unscheduled meetings',
       'Can I Call you three hours before the interview and follow up on your attendance for the interview',
       'Can I have an alternative number/ desk number. I assure you that I will not trouble you too much',
       'Have you taken a printout of your updated resume. Have you read the JD and understood the same',
       'Are you clear with the venue details and the landmark.',
       'Has the call letter been shared', 'Expected Attendance',
       'Observed Attendance', 'Marital Status'],
      dtype='object')

In [216]:
for column in interviews.columns:
    print("-- " + column)
    print(interviews[column].value_counts())


-- Date of Interview
06.02.2016               220
04/12/16                  86
13.04.2016                84
02.09.2016                43
25.06.2015                43
21.4.16                   38
22.01.2016                34
07.05.2016                31
03.04.2015                30
14.01.2016                28
27.02.2016                26
26.02.2016                26
05/11/2016                25
20.08.2016                25
25.5.2016                 24
23.02.2016                22
02.12.2015                19
19.06.2015                18
19.03.16                  17
30.01.2016                16
09-01-2016                15
13.06.2016                15
11.5.16                   14
21.4.2016                 14
25-5-2016                 14
30.1.16                   14
22.03.2016                13
13/03/2015                13
3.6.16                    12
25.05.16                  12
                        ... 
28.08.2016 & 11.30 AM      2
21/04/16                   2
28.08.2016 & 09.30 AM 

In [217]:
def parse_date(date_string):
    try:
        preprocessed = re.sub(r"[-–]", ' ', date_string)
        preprocessed = re.sub(r"\s{2,}", ' ', preprocessed)
        preprocessed = re.sub(r"(&.*)", ' ', preprocessed)
        return parse(preprocessed, ignoretz = True, dayfirst = True)
    except (ValueError, TypeError) as e:
        raise RuntimeError('Cannot be parsed as a date: ' + date_string) from e
        
        
assert parse_date("06.02.2016") == datetime.datetime(2016, 2, 6, 0, 0)
assert parse_date("04/12/16") == datetime.datetime(2016, 12, 4, 0, 0)
assert parse_date("21.4.16") == datetime.datetime(2016, 4, 21, 0, 0)
assert parse_date("15 Apr 16") == datetime.datetime(2016, 4, 15, 0, 0)
assert parse_date("09-01-2016") == datetime.datetime(2016, 1, 9, 0, 0)
assert parse_date("09 01 2016") == datetime.datetime(2016, 1, 9, 0, 0)
assert parse_date("25 – Apr-16") == datetime.datetime(2016, 4, 25, 0, 0)
assert parse_date("21-Apr -16") == datetime.datetime(2016, 4, 21, 0, 0)
assert parse_date("28.8.2016 & 12.00 PM") == datetime.datetime(2016, 8, 28, 0, 0)

In [282]:
class DateParser(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name = "Date of Interview"):
        self.attribute_name = attribute_name
        self.__days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        self.__months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        
        
    def __create_date_columns(self, value):
        date = parse_date(value)
        return pd.Series({
                "interview_month": self.__months[date.month - 1],
                "interview_day_of_week": self.__days_of_week[date.weekday()]
            })
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X[self.attribute_name].apply(
            lambda date: self.__create_date_columns(date)
        )
        dataframe = pd.concat([result, X], axis=1)
        return dataframe.drop([self.attribute_name], axis = 1)

In [219]:
def parse_city_name(city_name):
        preprocessed = re.sub(r"(\W)", '', city_name)
        return preprocessed.title()

class CityNameParser(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: parse_city_name(name)
        )
        return result
    
assert parse_city_name("Chennai") == "Chennai"
assert parse_city_name("- Cochin-") == "Cochin"
assert parse_city_name("CHENNAI") == "Chennai"
assert parse_city_name("chennai") == "Chennai"

In [220]:
class MergeITIndustries(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: "IT" if name.startswith("IT") else name
        )
        return result

In [221]:
#sorry, just gut feeling, due to the lack of a domain expert
class MergePositionType(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name
        self.mapping = {
            "Routine": "Routine",
            "Dot Net": "Routine",
            "Selenium testing": "Routine",
            "Niche": "Niche",
            "Trade Finance": "Niche",
            "AML": "Niche",
            "Production- Sterile": "Niche",
        }
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: self.mapping.get(name, "Routine")
        )
        return result

In [274]:
#sorry, just gut feeling, due to the lack of a domain expert
class MergeSkillsetType(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name
        
    def __map_skill(self, skill):
        name = skill.lower()
        if "java" in name:
            return "Java Developer"
        elif "testing" in name:
            return "Tester"
        elif "oracle" in name:
            return "Oracle"
        elif "aml" in name or "kyc" in name or "cdd" in name or "regulatory" in name:
            return "Regulatory"
        elif "sccm" in name:
            return "IT Infrastructure"
        elif "banking" in name or "lending" in name or name == "l & l":
            return "Banking Operations"
        elif "software engineer" in name or "developer" in name:
            return "Other software engineer"
        elif "lead" in name or "manager" in name or "management" in name:
            return "Management"
        elif "r & d" in name or "r&d" in name:
            return "R&D"
        elif "analyst" in name:
            return "Analyst"
        elif "publishing" in name:
            return "Publishing"
        elif "production" in name:
            return "Production"
        elif "fresher" in name:
            return "Fresher"
        elif "accounting operations" in name:
            return "Accounting Operations"
        elif "routine" in name:
            return "Routine"
        elif "sas" in name:
            return "SAS"
        elif "hadoop" in name:
            return "Hadoop"
        elif "dot net" in name:
            return ".NET"
        elif "etl" in name:
            return "ETL"
        elif "emea" in name:
            return "EMEA"
        elif "global labelling" in name:
            return "Global Labelling"
        elif "biosimiliars" in name:
            return "Biosimiliars"
        elif "product control" in name:
            return "Product Control"
        else:
            return "Other"
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: self.__map_skill(name)
        )
        return result

In [292]:
class ParseInterviewType(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name

    def __convert_interview_type(self, name):
        interview_type = name.lower()
        if "scheduled" in interview_type or "sceduled" in interview_type:
            return "Scheduled"
        else:
            return "Walk-in"
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: self.__convert_interview_type(name)
        )
        return result

In [309]:
class ParseYesNoAnswer(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name

    def __convert_answer(self, name):
        name = str(name).lower()
        if name == "yes":
            return "Yes"
        else:
            return "No"
            
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: self.__convert_answer(name)
        )
        return result

In [314]:
class ParseExpectedAttendanceAnswer(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_name):
        self.attribute_name = attribute_name

    def __convert_answer(self, name):
        name = str(name).lower()
        if name == "yes":
            return "Yes"
        elif name == "uncertain":
            return "Uncertain"
        elif name == "no":
            return "No"
        else:
            return "Yes"
            
    def fit(self, X, y = None):
        return self
    
    def transform(self, X):
        result = X.copy()
        result[self.attribute_name] = X[self.attribute_name].apply(
            lambda name: self.__convert_answer(name)
        )
        return result

In [317]:
preprocessing_pipeline = Pipeline([
    ("extract_interview_date", DateParser()),
    ("normalize_location", CityNameParser("Location")),
    ("normalize_candidate_current_location", CityNameParser("Candidate Current Location")),
    ("normalize_candidate_job_location", CityNameParser("Candidate Job Location")),
    ("normalize_interview_venue", CityNameParser("Interview Venue")),
    ("normalize_candidate_native_location", CityNameParser("Candidate Native location")),
    ("merge_it_industries", MergeITIndustries("Industry")),
    ("merge_position_type", MergePositionType("Position to be closed")),
    ("merge_skillset_type", MergeSkillsetType("Nature of Skillset")),
    ("parse_interview_types", ParseInterviewType("Interview Type")),
    ("convert_permission_answer", ParseYesNoAnswer("Have you obtained the necessary permission to start at the required time")),
    ("convert_unscheduled_meetings_answer", ParseYesNoAnswer("Hope there will be no unscheduled meetings")),
    ("convert_premeeting_call_answer", ParseYesNoAnswer("Can I Call you three hours before the interview and follow up on your attendance for the interview")),
    ("convert_alternative_number_answer", ParseYesNoAnswer("Can I have an alternative number/ desk number. I assure you that I will not trouble you too much")),
    ("convert_printout_answer", ParseYesNoAnswer("Have you taken a printout of your updated resume. Have you read the JD and understood the same")),
    ("convert_venue_answer", ParseYesNoAnswer("Are you clear with the venue details and the landmark.")),
    ("convert_call_letter_answer", ParseYesNoAnswer("Has the call letter been shared")),
    ("convert_expected_attendance_answer", ParseExpectedAttendanceAnswer("Expected Attendance")),
    ("convert_observed_attendance_answer", ParseYesNoAnswer("Observed Attendance")),
])

preprocessed = preprocessing_pipeline.transform(interviews)

-----------------------------------------------------------
-- interview_day_of_week
Saturday     423
Friday       215
Wednesday    192
Thursday     157
Sunday       110
Tuesday       94
Monday        42
Name: interview_day_of_week, dtype: int64
-----------------------------------------------------------
-- interview_month
February     311
April        246
May          152
January      122
June         121
December     105
March         63
August        45
September     43
November      25
Name: interview_month, dtype: int64
-----------------------------------------------------------
-- Client name
Standard Chartered Bank            904
Pfizer                              75
Hospira                             75
Aon Hewitt                          28
Flextronics                         23
ANZ                                 22
Hewitt                              20
UST                                 18
Prodapt                             17
Standard Chartered Bank Chennai     17
Astr